In [1]:
import os
import sys
import getpass
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, StateGraph, START
import functools
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS, Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores.faiss import DistanceStrategy

In [2]:
class ChatMessage(object):

    def __init__(self, message : BaseMessage, sender : str = None):

        self.message = message
        self.sender = sender

        self.content = message.content

    def __repr__(self) -> str:
        
        return f"{self.sender} | {self.content}"

def reducer(a : list, b : list | str ) -> list:

    if type(b) == list: 
        return a + b
    else:
        return a

    
class AgentState(TypedDict):
    messages: Annotated[Sequence[ChatMessage], reducer]

In [6]:
sys.path.insert(0,"/Users/roshansk/Documents/GitHub/Multilingual-Chatbot/")

from ml_chat.agents.agents import * 
from ml_chat.agents.workflow import *
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)

userAgent = UserAgent(llm, 'roshan', 'english')

In [8]:
data_directory = Path('/Users/roshansk/Documents/GitHub/Multilingual-Chatbot/ml_chat/playground/data')
retriever = load_retriever(data_directory, ['.pdf'])

In [10]:
ayaAgent = AyaQueryAgent(llm, retriever)
supervisorAgent = SupervisorAgent(llm)

In [14]:
aya_query_node = functools.partial(get_aya_query_node, agent = ayaAgent, supervisor_agent = supervisorAgent, name ="Aya_Query")
supervisor_node = functools.partial(get_supervisor_node, agent = supervisorAgent)
user_agent_node = functools.partial(get_user_node, agent=userAgent, name='roshan')

In [17]:
workflow = StateGraph(AgentState)
workflow.add_node("User", user_agent_node)
workflow.add_node("Supervisor", supervisor_node)
workflow.add_node("Aya_Query",aya_query_node)

In [18]:
workflow.add_edge(START, "Supervisor")

In [19]:
workflow.add_edge("Supervisor", 'User')
workflow.add_edge("User", END)

In [20]:
app = workflow.compile()

ValueError: Node 'Aya_Query' is a dead-end

In [37]:
path = '/Users/roshansk/Documents/GitHub/Perplexity-vs-Evaluation/Experiments/Perplexity_Runs/Data/samples_hellaswag_hi_2024-08-04T14-52-52.186900_no_template.jsonl'

ppl_path = '/Users/roshansk/Documents/GitHub/Perplexity-vs-Evaluation/Experiments/Perplexity_Runs/aya_23_8b_chat_ppl_hellaswag_with_okapi_with_template/aya-23-8B/samples_hellaswag_hi_chat_ppl_2024-08-08T16-09-09.035738.jsonl'


In [24]:
import json

def read_jsonl_file(path):
    data = []
    with open(path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

In [34]:
out = read_jsonl_file(path)

In [35]:
out

[{'text': 'Personal Care and Style: धनुष से अपनी फैशन वार्ड्रोब को कैसे अपडेट करें. ऐसी ब्लाउज खोजें जिनके कम से कम एक धनुष कहीं पर हो। यह टॉप के लिए सबसे आसान विकल्प है क्योंकि आपको अपने टॉप के रंग और साइज़ से मिलती-जुलती धनुष बनाने की चिंता नहीं करनी पड़ती।. धनुष मुद्रित ब्लाउज को एक विकल्प के रूप में विचार करें।'},
 {'text': 'Health: अपने बच्चे को दांत पिसने से कैसे रोकें. समझें कि तनाव दांत पिसने का कारण हो सकता है। दांत पिसाई, आपके बच्चे के स्ट्रेस या कुछ चिंता के बारे में परेशान होने का संकेत हो सकता है। अपने बच्चे को आराम दिलाने में मदद करना आपके बच्चे को दांत पिसने से रोकने की कुंजी हो सकता है।'},
 {'text': 'Pets and Animals: अपने बच्चों को कुत्ता चुनने में शामिल करने के लिए कैसे जोड़ें. एक कुत्ते को पाने की संभावना लाओ। अधिकांश बच्चों के लिए कुत्ता पाने का विचार बहुत उत्साहजनक होगा। अपने बच्चों के साथ इस मुद्दे को उठाएं जब वे उत्साहित होने और प्रश्न पूछने के लिए समय हो।'},
 {'text': 'High jump: आदमी एक कोण पर दौड़ता है और पीछे की ओर कूदकर छः फुट की बार को पार कर देता है। बाद म

In [38]:
ppl_result = read_jsonl_file(ppl_path)

In [39]:
ppl_result

[{'doc_id': 0,
  'doc': {'text': 'Personal Care and Style: धनुष से अपनी फैशन वार्ड्रोब को कैसे अपडेट करें. ऐसी ब्लाउज खोजें जिनके कम से कम एक धनुष कहीं पर हो। यह टॉप के लिए सबसे आसान विकल्प है क्योंकि आपको अपने टॉप के रंग और साइज़ से मिलती-जुलती धनुष बनाने की चिंता नहीं करनी पड़ती।. धनुष मुद्रित ब्लाउज को एक विकल्प के रूप में विचार करें।'},
  'target': '<BOS_TOKEN><|START_OF_TURN_TOKEN|><|USER_TOKEN|>Personal Care and Style: धनुष से अपनी फैशन वार्ड्रोब को कैसे अपडेट करें. ऐसी ब्लाउज खोजें जिनके कम से कम एक धनुष कहीं पर हो। यह टॉप के लिए सबसे आसान विकल्प है क्योंकि आपको अपने टॉप के रंग और साइज़ से मिलती-जुलती धनुष बनाने की चिंता नहीं करनी पड़ती।. धनुष मुद्रित ब्लाउज को एक विकल्प के रूप में विचार करें।<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>',
  'arguments': {'gen_args_0': {'arg_0': '<BOS_TOKEN><|START_OF_TURN_TOKEN|><|USER_TOKEN|>Personal Care and Style: धनुष से अपनी फैशन वार्ड्रोब को कैसे अपडेट करें. ऐसी ब्लाउज खोजें जिनके कम से कम एक धनुष कहीं पर हो। यह टॉप के लिए

In [41]:
import pandas as pd
df = pd.read_csv('/Users/roshansk/Documents/GitHub/Perplexity-vs-Evaluation/Data/HellaSwag_Consolidated/Hindi_8b_chat.csv')




In [42]:
df.head()

,acc,acc_norm,log_likelihood,num_words,num_bytes,num_bits_per_byte,word_perplexity,byte_perplexity
0,1.0,0.0,-356.463562,61,747,747,345.041587,1.611545
1,0.0,0.0,-299.411926,58,702,702,174.561059,1.531906
2,1.0,1.0,-294.324005,52,667,667,287.170760,1.554673
3,0.0,1.0,-178.408096,24,224,224,1692.007004,2.217687
4,0.0,1.0,-291.298431,42,555,555,1028.315122,1.690226
